_DISCLAIMER: this chapter and onwards is getting really stale. I wont be doing any practical stuff here. Instead i will approach Policy Gradient (PG) from point of view of Deep RL._

*   We learn policy $\pi(a|s, \vec{\theta}) = Pr\{A_t = a | S_t = s, \vec{\theta}_t = \vec{\theta}\}$ parameterized by $\vec{\theta} \in \R^{d^\prime}$. 

*   This approach allows us to avoid learning state value function, but it will be also useful. So we define it as $\hat v(s,\vec{w})$ with parameters in $\vec{w} \in \R^d$

*   Policy is improve by _policy gradient method_, which is about maximizing performance measure $J(\vec{\theta})$:
    $$\vec{\theta}_{t+1} = \vec{\theta}_t + \alpha \widehat{\nabla J(\vec{\theta}_t)}$$
    where $\widehat{\nabla J(\vec{\theta}_t)}$ is a stochastic estimate of performance.

*   in case both policy and value function is learned, then method is classified as _actor-critic method_ with actor and critic learning respective aspects.



For discrete actions we can learn _preference_ $h(s,a,\vec{\theta}) \in \R$, and obtain policy by applying _soft-max_:
$$\pi(a|s, \vec{\theta}) = \frac{e^{h(s,a,\vec{\theta})}}{\sum_b e^{h(s,b,\vec{\theta})}}$$
Or by linear approximation:
$$h(s,a,\vec{\theta}) = \vec{\theta}^T \vec{x}(s,a)$$
with feature vector $\vec{x}(s,a) \in \R^{d^\prime}$

Benefits:
*   soft-max action preference policy can approach deterministic, unlike $\epsilon$- greedy approach.<br>_Rate of convergence can be controlled by 'temperature' parameter._
*   soft-max approach can create policies which are stochastic;
*   policy may be simpler to approximate;
*   can inject prior knowledge in the system via policy


# The Policy Gradient Theorem

Lets define policy performance metric as a true value function of a policy:
$$J(\vec{\theta}) = v_{\pi_\theta}(s_0)$$
*    performance metric will depends, in part, by state distribution -  how often states are visited. <br>This aspect depends on the environment and should be considered unknown.

Policy gradient theory shows that for episodic tasks:
$$\nabla J(\vec{\theta}) \propto \sum_s \mu(s)\sum_a q_\pi (s,a) \nabla \pi(a|s,\vec{\theta})$$
with $\mu$ being on-policy distribution under $\pi$

## REINFORCE: Monte Carlo Policy Gradient

$$\nabla J(\vec{\theta}) \propto \sum_s \mu(s)\sum_a q_\pi (s,a) \nabla \pi(a|s,\vec{\theta}) = \mathbb{E}_\pi \bigg[\sum_a q_\pi (S_t,a) \nabla \pi(a|S_t,\vec{\theta})\bigg]$$
so its possible to create an update rule:
$$\vec{\theta}_{t+1}  = \vec{\theta}_{t} + \alpha \sum_a \hat q_\pi (S_t,a) \nabla \pi(a|S_t,\vec{\theta})$$
It involves taking all actions, while we usually have access only to some known action $A_t$.

To be a true expectation OG expression is missing weighting $\pi(a|S,\vec{\theta})$. By multiplying expression by $\frac{\pi(a|S,\vec{\theta})}{\pi(a|S,\vec{\theta})} = 1$ we get:
$$\nabla J(\vec{\theta}) \propto \mathbb{E}_\pi \bigg[\sum_a \pi(a|S,\vec{\theta}) \cdot q_\pi (S_t,a) \frac{\nabla \pi(a|S_t,\vec{\theta})}{\pi(a|S,\vec{\theta})}\bigg]$$
$$ = \mathbb{E}_\pi \bigg[q_\pi (S_t,A_t) \cdot \frac{\nabla \pi(A_t|S_t,\vec{\theta})}{\pi(A_t|S,\vec{\theta})}\bigg]$$
$$ = \mathbb{E}_\pi \bigg[G_t \cdot \frac{\nabla \pi(A_t|S_t,\vec{\theta})}{\pi(A_t|S,\vec{\theta})}\bigg]$$

thus,
$$\vec{\theta}_{t+1}  = \vec{\theta}_{t} + \alpha G_t \frac{\nabla \pi(A_t|S_t,\vec{\theta})}{\pi(A_t|S,\vec{\theta})}$$

Interpretation is intuitive:<br>
$G_t$ scales vector of steepest ascent in parameter space $\nabla \pi(A_t|S_t,\vec{\theta})$ and denominator scales it down based on probability of taking this action.

We can write right term (called _eligibility vector_) more compactly via:
$$\nabla ln \ \pi(A_t|S_t, \vec{\theta}) = \frac{\nabla \pi(A_t|S_t,\vec{\theta})}{\pi(A_t|S,\vec{\theta})}$$

Version with _baseline_:
$$\vec{\theta}_{t+1}  = \vec{\theta}_{t} + \alpha \bigg(G_t - b(S_t)\bigg) \frac{\nabla \pi(A_t|S_t,\vec{\theta})}{\pi(A_t|S,\vec{\theta})}$$